In [1]:
import os

from keras.layers import Input
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

from FullPointNet import FullPointNet
from losses import level_loss
from puppet_dataset_keypoint.puppet import PuppetDataset
from DataGenerator import DataGenerator
from EvalPuppet import EvalPuppet
from TestDataGenerator import TestDataGenerator
from visualize import display_my_masks

Using TensorFlow backend.


In [2]:
TRAINING_LOG = os.path.join('logdir', "training.csv")

def create_model():
    input_image = Input((None, None, 3))
    return FullPointNet(input_image)

# create the model
model = create_model()
print('model created!')

model created!


In [3]:
train_batch_size = 6
test_batch_size = 6
height = 48
width = 48
epoch_samples = 6

In [4]:
# create a generator for training data
train_generator = DataGenerator(PuppetDataset, kpn=4,
                                batch_size=train_batch_size,
                                height=height, width=width)
test_generator = TestDataGenerator(PuppetDataset, kpn=4,
                                   batch_size=test_batch_size,
                                   height=height, width=width)

In [5]:
# first stage training
model.compile(loss={'p1': level_loss(), 'p2': level_loss(), 'p3': level_loss()},
              loss_weights={'p1': 0.7, 'p2': 0.2, 'p3': 0.1},
              optimizer=adam(lr=1e-2))
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=epoch_samples // train_batch_size,
    epochs=20,
    callbacks=[
        ModelCheckpoint(os.path.join('savedir', 'best.h5'), monitor='loss',
                        verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1,
                          mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
        CSVLogger(TRAINING_LOG, append=True),
    ]
)

Epoch 1/20
Epoch 00001: loss improved from inf to 2.72398, saving model to savedir/best.h5
Running keypoint evaluation: gt 6 images; dt 6 images
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
DONE (t=0.00s).
oks threshold: [ 0.5   0.55  0.6   0.65  0.7   0.75  0.8   0.85  0.9   0.95]
corresponding AP: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
mean AP: 0.0
1/1 [==============================] - 30s 30s/step - loss: 2.7240 - p1_loss: 2.9143 - p2_loss: 2.3823 - p3_loss: 2.0751
Epoch 2/20
Epoch 00002: loss did not improve
Running keypoint evaluation: gt 6 images; dt 6 images
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
DONE (t=0.00s).
oks threshold: [ 0.5   0.55  0.6   0.65  0.7   0.75  0.8   0.85  0

/root/JupyterNotebook/fullpointnet/puppet_dataset_keypoint/eval.py:70: RuntimeWarning: invalid value encountered in true_divide
  result = sum_match / float(sum_object)


Epoch 00009: loss did not improve
Running keypoint evaluation: gt 6 images; dt 6 images
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: ground truth object 0 in all images!
DONE (t=0.00s).
oks threshold: [ 0.5   0.55  0.6   0.65  0.7   0.75  0.8   0.85  0.9   0.95]
corresponding AP: [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
mean AP: nan
1/1 [==============================] - 8s 8s/step - loss: 1.5137 - p1_loss: 1.4750 - p2_loss: 1.5748 - p3_loss: 1.6620
Epoch 10/20
Epoch 00010: loss did not improve

Epoch 00010: reducing learning rate to 0.0009999999776482583.
Running keypoint evaluation: gt 6 images; dt 6 images
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detection object in one image
Wanrning: no detecti

In [ ]:
def compare():
    # build dataset
    batch_size = 1
    height = 48
    width = 48
    dataset = TestDataGenerator(PuppetDataset, 4, batch_size, height=height,
                                width=width)
    # generate data
    image_group, guide_mask_group, annkp_group = dataset.next()
    predict_mask_group = model.predict_on_batch(image_group)[-1]
    for x in range(batch_size):
        image = image_group[x]
        # select last level see, and unuse other level
        true_mask = guide_mask_group[x][-1]
        display_my_masks(image, true_mask)
        # select last level see, and unuse other level
        predict_mask = predict_mask_group[x, :, :, :]
        display_my_masks(image, predict_mask)

In [6]:
# second stage training
model.compile(loss={'p1': level_loss(), 'p2': level_loss(), 'p3': level_loss()},
              loss_weights={'p1': 0.33, 'p2': 0.33, 'p3': 0.33},
              optimizer=adam(lr=1e-3))
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=epoch_samples // train_batch_size,
    epochs=20,
    callbacks=[
        ModelCheckpoint(os.path.join('savedir', 'best.h5'), monitor='loss',
                        verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1,
                          mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
        CSVLogger(TRAINING_LOG, append=True),
        EvalPuppet(test_generator, batch_size=test_batch_size)
    ]
)

Epoch 1/20
 18/200 [=>............................] - ETA: 16:44 - loss: 319.9221 - p1_loss: 467.0486 - p2_loss: 284.3077 - p3_loss: 218.1047

KeyboardInterrupt: 

In [ ]:
# third stage training
model.compile(loss={'p1': level_loss(), 'p2': level_loss(), 'p3': level_loss()},
              loss_weights={'p1': 0.1, 'p2': 0.2, 'p3': 0.7},
              optimizer=adam(lr=1e-4))
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=epoch_samples // train_batch_size,
    epochs=20,
    callbacks=[
        ModelCheckpoint(os.path.join('savedir', 'best.h5'), monitor='loss',
                        verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1,
                          mode='auto', epsilon=0.0001, cooldown=0, min_lr=0),
        CSVLogger(TRAINING_LOG, append=True),
        EvalPuppet(test_generator, batch_size=test_batch_size)
    ]
)

In [ ]:
# store final result too
model.save(os.path.join('savedir', 'final.h5'))